# Hive

### Мотивация 
- Написание Hadoop-задач на Java нетривиально
- Хочется использовать возможность Hadoop для обработки слишком больших для обычной реляционной БД данных
- Хочется иметь определенную схему данных
- SQL? Да - HiveSQL

### Архитектура

![Hive](https://cwiki.apache.org/confluence/download/attachments/27362072/system_architecture.png)

- `Metastore` хранит метаинформацию о таблицах, колонках и их типах, где и как они хранятся и т.д.
    - В качестве хранилища может использоваться реляционная БД
- `Driver`- компонент, который обслуживает жизненный цикл запроса к `Hive`.
- `Query Compiler` - компонент, который обрабатывает запроса на `HiveSQL` и преобразует его в последовательность MapReduce-задач
- `Execution Engine` - компонент, который непосредственно запускает MapReduce-задачи. 

### Клиент Python


```bash
python3 -m pip install pyhive[hive_pure_sasl]
```

In [5]:
from pyhive import hive  
with hive.connect('localhost', username='root') as conn:
    with conn.cursor() as cursor:        
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS person (               
                n STRING, 
                age INT
            )
            ''')
        
    with conn.cursor() as cursor:
        cursor.execute('''
            INSERT INTO TABLE person VALUES ("John2", 550), ("Mike2", 800)
        ''')
     
    with conn.cursor() as cursor:
        cursor.execute('''
            SELECT n, count(*) FROM person group by n
        ''')
        
        print(cursor.fetchall())


[('John2', 1), ('Mike2', 1)]


### Хранение данных

- База Данных - это набор Hive-таблиц
    - физически представляет собой директорию на HDFS
- Таблицы в базе данных 
    - таблицы физически хранятся в поддиректориях
    - метаинформация о таблице хранится в `Metastore`

Создать базу дынных:
```sql 
CREATE DATABASE test_db;
```
   
Создать таблицу:
```sql
CREATE TABLE weather ( 
               dt TIMESTAMP, 
               t FLOAT, 
               po FLOAT,
               p FLOAT, 
               u FLOAT, 
               vv FLOAT, 
               td float, 
               n STRING) 
           ROW FORMAT DELIMITED FIELDS TERMINATED BY ','; 
               
LOAD DATA LOCAL INPATH '/course/data/weather_stat.csv' INTO TABLE weather;               
```


```sql
CREATE TABLE cik ( 
               region STRING, 
               tik STRING, 
               uik STRING, 
               voters_total INT,
               total INT, 
               total_ahead INT,
               total_inside INT, 
               total_outside INT, 
               total_removed INT,
               outside INT,
               inside INT, 
               invalid INT,
               valid INT, 
               lost INT,
               unkwn INT,
    
               baburin INT,
               grudinin INT, 
               zhirinovsky INT, 
               putin INT, 
               sobchak INT,
               suraykin INT,
               titov INT, 
               yavlinsky INT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','; 
               
LOAD DATA LOCAL INPATH '/course/data/cik_trunc.csv' INTO TABLE cik;               
```

### Типы данных

##### Numeric Types

    TINYINT (1-byte signed integer)
    SMALLINT (2-byte signed integer)
    INT/INTEGER (4-byte signed integer)
    BIGINT (8-byte signed integer)
    FLOAT (4-byte single precision floating point number)
    DOUBLE (8-byte double precision floating point number)
    DECIMAL
    NUMERIC 

##### Date/Time Types

    TIMESTAMP 
    DATE 
    INTERVAL

##### String Types

    STRING
    VARCHAR 
    CHAR 

##### Misc Types

    BOOLEAN
    BINARY 

##### Complex Types
    
    arrays: ARRAY<data_type> 
    maps: MAP<primitive_type, data_type> 
    structs: STRUCT<col_name : data_type [COMMENT col_comment], ...>
    union: UNIONTYPE<data_type, data_type, ...> 



### Создание таблиц

Колонки можно разделять с помощью регулярных выражений

```sql
CREATE TABLE apachelog (
      host STRING,
      identity STRING,
      user STRING,
      time STRING,
      request STRING,
      status STRING,
      size STRING,
      referer STRING,
      agent STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
  "input.regex" = "([^]*) ([^]*) ([^]*) (-|\\[^\\]*\\]) ([^ \"]*|\"[^\"]*\") (-|[0-9]*) (-|[0-9]*)(?: ([^ \"]*|\".*\") ([^ \"]*|\".*\"))?"
)
STORED AS TEXTFILE;
```

Для кастомного CSV

```sql
CREATE TABLE my_table(a string, b string, ...)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   "separatorChar" = "\t",
   "quoteChar"     = "'",
   "escapeChar"    = "\\"
)  
```

### Partitioning

Физически партиции хранятся в разных директориях

```sql 

CREATE TABLE page_view(viewTime INT, userid BIGINT,
     page_url STRING, referrer_url STRING,
     ip STRING COMMENT 'IP Address of the User')
 COMMENT 'This is the page view table'
 PARTITIONED BY(dt STRING, country STRING)
 STORED AS SEQUENCEFILE;
```

Можно использовать динамическое партиционирование 

```sql
SET hive.exec.dynamic.partition.mode=nonstrict;

CREATE TABLE cik_ext(
    uik string, 
    valid INT, 
    putin INT, 
    grudinin INT) 
 partitioned BY (region string);
 
INSERT overwrite TABLE cik_ext partition(region) 
SELECT uik , valid, putin, grudinin, region
FROM cik;
```

### Skewed tables

```sql
CREATE TABLE skewed_table (col1 STRING, col2 INT, col3 STRING)
  SKEWED BY (col1, col2) ON (('s1',1), ('s3',3), ('s13',13), ('s78',78)) STORED AS DIRECTORIES;
```                                                              

### Bucketed table

```sql
CREATE TABLE page_view(
    viewTime INT, 
    userid BIGINT,
    page_url STRING, 
    referrer_url STRING,
    p STRING) 
 PARTITIONED BY(dt STRING, country STRING)
 CLUSTERED BY(userid) SORTED BY(viewTime) INTO 32 BUCKETS
 STORED AS SEQUENCEFILE;
 ```
 

### Загрузка данных

```sql
LOAD DATA LOCAL INPATH '/page_views.csv'
OVERWRITE INTO TABLE page_view
PARTITION (dt='2017-05-05', country='RU');
```

### Insert

```sql

CREATE TABLE pageviews (
    userid VARCHAR(64), 
    link STRING, 
    came_from STRING
  ) 
  PARTITIONED BY (datestamp STRING) 
  CLUSTERED BY (userid) INTO 256 BUCKETS 
  STORED AS ORC;
 
INSERT INTO TABLE pageviews PARTITION (datestamp = '2014-09-23')
  VALUES ('jsmith', 'mail.com', 'sports.com'), 
         ('jdoe', 'mail.com', null);
 
INSERT INTO TABLE pageviews PARTITION (datestamp)
  VALUES ('tjohnson', 'sports.com', 'finance.com', '2014-09-23'), 
         ('tlee', 'finance.com', null, '2014-09-21');
         
         
         
FROM old_pageviews
  INSERT OVERWRITE TABLE pageviews_1 PARTITION (datestamp = '2014-09-23')
      SELECT * WHERE came_from='google.com'
  INSERT OVERWRITE TABLE pageviews_2 PARTITION (datestamp = '2014-09-23')
      SELECT * WHERE came_from='yandex.ru';         
  
```

### Views


```sql
CREATE VIEW onion_referrers(url)
  AS
  SELECT DISTINCT referrer_url
  FROM page_view
  WHERE page_url='http://www.example.com';
```

### Joins

```sql

CREATE TABLE a (k1 string, v1 string);
CREATE TABLE b (k2 string, v2 string);

SELECT k1, v1, k2, v2
FROM a JOIN b ON k1 = k2; 

SELECT a.* FROM a JOIN b ON (a.id = b.id AND a.department = b.department)

```

### UDF

Функции в `HiveSQL`, которые могут определять пользователи:

`our_udf.py`
```python 

import sys

# оставляем только последнее слово
for line in sys.stdin:
    for val in line.strip().split('\t'):
        print(val.split(' ')[-1])
```


В консоли пишем:
```sql
ADD FILE our_udf.py;

SELECT TRANSFORM(region, uik) USING 'python3 out_udf.py' AS (h STRING) FROM cik;

```